In [4]:
%run '../common/utils.ipynb'

from keras.callbacks import TensorBoard

def signature(inputs, outputs):
    return (
      tf.saved_model.signature_def_utils.predict_signature_def(
          inputs={'images': inputs},
          outputs={'scores': outputs}))

def signature_def_map(signature):
    return {'predict': signature}

import tensorflow as tf

class BatchLevelTensorBoard(TensorBoard):
    '''
    Batch Level Metrics
    '''
    def __init__(self, log_every=1, **kwargs):
        super().__init__(**kwargs)
        self.log_every = log_every
        self.counter = 0
        
    def on_batch_end(self, batch, logs=None):
        self.counter+=1
        if self.counter%self.log_every==0:
            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, self.counter)
            self.writer.flush()
        
        super().on_batch_end(batch, logs)